#Se importan las librerias

In [2]:
!pip install tensorflow==1.14.0

In [0]:
import zipfile
import os 
import math
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt 
from sklearn import preprocessing, model_selection
from keras.utils import to_categorical 
import keras
from keras import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D  

#Se cargan los datos

In [0]:
mundo1 = zipfile.ZipFile("Mun1.zip", mode="r")
mundo1.extractall(None)
mundo8 = zipfile.ZipFile("Mun8.zip", mode="r") 
mundo8.extractall(None)

#Se definen las variables 'X' y 'y'

In [6]:
imagenes = []
nivel = []
imagenes1 = mundo1.namelist()
imagenes8 = mundo8.namelist()
#Se agregan las imagenes del mundo 1
i = 1
while(i<len(imagenes1)):
  filepath = os.path.join("", imagenes1[i]) 
  image = plt.imread(filepath)
  imagenes.append(image)
  nivel.append(0)
  i += 1
#Se agreagan las imagenes del mundo 8
i = 1
while(i<len(imagenes8)):
  filepath = os.path.join("", imagenes8[i]) 
  image = plt.imread(filepath)
  imagenes.append(image)
  nivel.append(1)
  i += 1
#Se define la salida por categorias, para que se puedan distinguir las clases.
y = to_categorical(np.array(nivel))
X = np.array(imagenes, dtype=np.uint8)  
print('Dimensiones de las variables: ', X.shape, y.shape)

Dimensiones de las variables:  (16435, 200, 240, 3) (16435, 2)


#Preprocesamiento de datos


1.   Se realiza el conjunto de entrenamiento al tomar el 10% de los datos totales.
2.   Se realiza el conjunto de validación al tomar el 10% de los datos de entrenamiento.



In [0]:
#Se definen los porcentajes de entrenamiento y validación
porcentajeEntrenamiento = 0.8
porcentajeValidación = 0.9
#Se determinan los datos de entrenamiento, prueba y validación
x_entrenamiento, x_prueba, y_entrenamiento, y_prueba = model_selection.train_test_split(X,y,test_size=1-porcentajeEntrenamiento)
x_entrenamiento, x_validacion, y_entrenamiento, y_validacion = model_selection.train_test_split(x_entrenamiento,y_entrenamiento,test_size=1-porcentajeValidación)

#Se crea la red neuronal


1.   La red neuronal se realizará a partir de keras
2.   La red va a ser de tipo sequencial, para poder trabajar con diferentes capas, que se encarguen de diferentes funciones.
3.   La primera capa es de tipo Conv2D, esto con el fin de especificar que se esta recibiendo una imagen de dimenciones (200,240,3). Tambien se especifica que su activación es de tipo relu, ya que siempre se recomienda hacer pooling justo despues de haber aplicado una capa con no-linealidad.
4.   Lo siguiente es utilizar MaxPooling2D, para agrupar la presencia de aquellos patrones que se repiten con mayor frecuencia en cada segmento de la imagen.
5.   A continuación se utiliza Dropout, para igualar a 0 un pequeño porcentaje de la información y con esto evitar overfitting en el entrenamiento.
6.   Se utiliza flatten para acoplar los datos que salen de la capa dentreada y ajustarlos para que sirvan como entrada de la capa escondida de clasificación que se utiliza.
7.   Se establece una capa central para para clasificación y se utiliza una activacion tipo relu, ya que sabemos suele entregar los mejores resultados en clasificación.
8.   Nuevamente utilizamos Dropout para evitar overfitting.
9.   Ya para terminar se crea una capa se salida con activación softmax para comprimir los nCentral datos obtenidos por la capa de clasificación, en tan solo nFinal datos que corresponderan al npumero de salidas.
10.  Por ultimo se realiza la compilación de la red, la cual cuenta con:
      1.  Entropia cruzada como función de error, dado que estamos haciendo un clasificador.
      2.  Descenso de gradiente estocastico como algoritmo de optimización.
      3.  La tasa de aprendisaje se fijo en 0.0005, luego de varias pruebas.
      4.  Ademas tambien se indica que debe retornar el porcentaje de aciertos del entrenamiento.


In [0]:
#Función que crea la red neuronal con base en el número de neuronas de cada una de sus capas
def crearRed(nInicial, nCentral, nSalida):
  #Se crea la red sequencial
  RedNeuronal = Sequential()
  #Se crea la capa de entrada
  RedNeuronal.add(Conv2D(nInicial, kernel_size=(3, 3),activation='relu',padding='same',input_shape=(200, 240, 3)))
  #Se realiza una agrupación de datos
  RedNeuronal.add(MaxPooling2D())
  RedNeuronal.add(MaxPooling2D())
  RedNeuronal.add(MaxPooling2D())
  RedNeuronal.add(MaxPooling2D())
  RedNeuronal.add(MaxPooling2D())
  #Se ajusta los datos para conectarlos a la siguiente capa
  RedNeuronal.add(Dropout(0.5))
  RedNeuronal.add(Flatten())
  #Se crea la capa central de clasificación
  RedNeuronal.add(Dense(nCentral, activation='relu'))
  RedNeuronal.add(Dense(nCentral, activation='relu')) 
  RedNeuronal.add(Dropout(0.5)) 
  #Se crea la capa de salida
  RedNeuronal.add(Dense(nSalida, activation='softmax'))
  #Se compila la red
  tasa = 0.0005
  RedNeuronal.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.SGD(lr=tasa,decay=tasa/100), metrics=['accuracy'])
  #Se retorna la red creada
  return RedNeuronal

#Validación de número de neuronas


1.   Se van a realizar pruebas de 10 en 10 en cada una de las capas.
2.   Dado que nuestra variable de salida esta restringida a dos posibles salidas debido a la funcion to_categorical(), la ultima capa tambien debera ser de dos salidas, para que la respuesta se pueda comparar con los datos reales.



In [11]:
#Se crea el vector con el número de neuronas
neuronas = [30, 40, 50]
i = 0
#Se recorre la capa de entrada
while(i<len(neuronas)):
  j=0
  #Se recorre la capa central
  while(j<len(neuronas)):
    #Se crea y valida la red neuronal
    RedNeuronal = crearRed(neuronas[i], neuronas[j], 2)
    fit = RedNeuronal.fit(x_validacion, y_validacion, batch_size=64, epochs=10, verbose=0)
    print('[',neuronas[i],',',neuronas[j],', 2 ]:','Tasa de acierto: ',(fit.history['acc']))
    j += 1
  i += 1

[ 30 , 30 , 2 ]: Tasa de acierto:  [0.4714828905043946, 0.50114068456929, 0.5003802282501989, 0.5262357420341144, 0.537642586185905, 0.5346007612268281, 0.5110266160375718, 0.5520912547755151, 0.5292775673104783, 0.5239543727822177]
[ 30 , 40 , 2 ]: Tasa de acierto:  [0.5079847915544256, 0.4996197719764347, 0.5057034220758952, 0.5277566540150588, 0.5391634987334335, 0.5718631183693165, 0.5186311791604916, 0.5361216730264656, 0.582509506202016, 0.5543726235968078]
[ 30 , 50 , 2 ]: Tasa de acierto:  [0.5057034227331328, 0.501901140707074, 0.5193916350036519, 0.5285171109913873, 0.5026615975927491, 0.5140684413139358, 0.5239543726915642, 0.5155893537254841, 0.5384030418930852, 0.5338403049077372]
[ 40 , 30 , 2 ]: Tasa de acierto:  [0.5049429664593685, 0.5361216736837032, 0.5346007609548677, 0.5117870730139004, 0.5110266160375718, 0.5178707225014502, 0.5239543733941285, 0.5285171103341498, 0.5361216736837032, 0.5627376430841453]
[ 40 , 40 , 2 ]: Tasa de acierto:  [0.5072243353259881, 0.508

#Se selecciona la red neuronal a crear

1.   Se va a tomar la red con las neuronas [50, 50, 2], dado que se obxervaron muy buenos resultados para este tipo de red.
2.   Se entrena la red con los datos de en trenamiento y esto luego de 5 epochs.
3.   Se guarda la red obtenida.



In [10]:
#Se crea la red neuronal
RedNeuronal = crearRed(50, 50, 2)
#Se muestra como esta compuesta la red
RedNeuronal.summary()
#Se entrena la red
fit = RedNeuronal.fit(x_entrenamiento, y_entrenamiento, batch_size=64, epochs=15, verbose=1)
#Se guarda la red creada
RedNeuronal.save("CasoSencillo.h5py")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 200, 240, 50)      1400      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 100, 120, 50)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 50, 60, 50)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 25, 30, 50)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 12, 15, 50)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 6, 7, 50)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 6, 7, 50)         

KeyboardInterrupt: ignored

#Evaluación de la red neuronal
Una vez evaluado el error empirico y rteniendo en cuenta el número de datos \\
Se observa que el indice de confianza es :
##$(1-\delta) = (1 - 2/\exp(|S{\small text}|{ \cdot 2\varepsilon^2 }))$

In [14]:
#Se prueba la red neuronal
prueba = RedNeuronal.evaluate(x_prueba, y_prueba, verbose=0)
#Se calcula el erro empirico, numero de datos y indice de confianza
E_empirico = 1 - prueba[1]
N_datos = y_prueba.shape[0]
confianza = 1 - 2/math.exp(N_datos*2*(E_empirico**2))
RedNeuronal.save("/content/drive/My Drive/Archivos MachineLearning/CasoSencillo.h5py")
print('Número de datos de prueba = ' + str(N_datos))
print('Error empirico = ' + str(E_empirico))
print('indice de confianza = ' + str(confianza))

Número de datos de prueba = 3287
Error empirico = 0.031335564271853134
indice de confianza = 0.9968551184248527


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#Referencias


*   [1]   "Home - Keras Documentation", Keras.io, 2020. [Online]. Available: https://keras.io/. [Accessed: 20- Mar- 2020].
*   [2]   "Clasificación de Imágenes en Python", Aprende Machine Learning, 2020. [Online]. Available: https://www.aprendemachinelearning.com/clasificacion-de-imagenes-en-python/. [Accessed: 20- Mar- 2020].



